# Capstone Project - Final
### Weeks 4 & 5

## Table of contents
* [Introduction: Business Problem](#business_problem)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Business Problem <a name="business_problem"></a>

The goal of this project is to find set of optimal locations for tourist accomodation (hotel, motel, inn, bed & breakfast etc) in the city of Ljubljana, capital of Slovenia.  

Ljubljana has been a very popular tourist destination in the past years and it hosts more tourists each year. The accomodation capacities need to grow accordingly. 

The following assumptions will be considered when trying to find optimal locations for accomodation:
* No other accomodation object in vicinity.
* Migrating towards to city center (where most of the city attractions are)
* Walking distance to various food places. 
* Walking distance to bus stops.

This information is interesting to investors and stakeholders that are considering new acommodation objects in Ljubljana. Location information is one of the most important variables in this decision. There are other factors that impact the decision which are not considered in this project, like realestate prices. 

## Data <a name="data"></a>

The source of data will be Foursquare:
* Existing accomodation locations in Ljubljana
* Food venues available walking distance from every accomodation location
* Bus stops locations available walking distance from every accomodation location

Existing administrative regions neighbourhoods/boroughs/districts could be used as originating points for Foursquare data. Unfortunately they have incorrect shapes which can skew the data pulled from Foursquare.
It was decided to put a square grid over the city of Ljubljana, centered at the most popular tourist spot, Ljubljansko tromostovje.

For address Nomatim is used.

In [4]:
# imports

# uncomment to install with conda or pip
# !pip install geopy
# !conda install -c conda-forge geopy --yes 

from geopy.geocoders import Nominatim 
from geopy.extra.rate_limiter import RateLimiter

# !conda install -c conda-forge folium --yes
# !pip install folium
import folium

import requests # library to handle requests
import pandas as pd
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import numpy as np
import math
import time

# !conda install -c conda-forge pyproj --yes 
import pyproj

In [20]:
# foursquare credentials
foursquare_client_id = 'EXENEWRKIY0ZQPXVOZCD3RWEXTNNQC1113GGBXYXJVHERV0J' # your Foursquare ID
foursquare_client_secret = 'YHAQNOL1EMK0BHONYERC4J52J4ECBPAIQOKWSULTVWZZUQKM' # your Foursquare Secret
foursquare_version = '20180605' # Foursquare API version
foursquare_limit = 100

### Accomodation locations

In [5]:
# Get the Ljubljana tourist center point.

lj_center_address = 'Prešernov trg, Ljubljana, Slovenia'

geolocator = Nominatim(user_agent="LJ_explorer")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=5)

location = geocode(lj_center_address)
lj_center_lat = location.latitude
lj_center_lon = location.longitude
lj_center_coordinates=(lj_center_lat,lj_center_lon)

print('Coordinates of {} = {}'.format(lj_center_address, lj_center_coordinates))

Coordinates of Prešernov trg, Ljubljana, Slovenia = (46.05140755, 14.506095911950972)


In [17]:
# Helper methods for calculating various geo coordinates in WGS84 space 

geod = pyproj.Geod(ellps='WGS84')

# calculate the geo point which is distance away in the direction of the fwd_azimuth
def calculate_geo_point(start_lat, start_lon, fwd_azimuth, distance):
    end_lon, end_lat, back_azimuth = geod.fwd(start_lon,start_lat,fwd_azimuth,distance)
    return (end_lat, end_lon)

# calculates geo coordinates of corners of a square. Start_lat and start_lon assumes SW corner.
def calculate_square_corners(start_lat, start_lon, distance):
    corners = [] # clock wise - sw, nw, ne, sw
    corners.append((start_lat, start_lon))
    corners.append(calculate_geo_point(start_lat, start_lon, 0, distance))
    corners.append(calculate_geo_point(start_lat, start_lon, 45, math.sqrt(2)*distance))
    corners.append(calculate_geo_point(start_lat, start_lon, 90, distance))
    return corners

# calculate distance between 2 geo coordinates 
def calculate_distance(start_lat, start_lon, end_lat, end_lon):
    a1, a1, distance = geod.inv(start_lon, start_lat, end_lon, end_lat)
    return distance

In [27]:
# Define the grid - a square defined by coordinates of it's corners (sw, nw, ne, se).
# Each grid unit/area is a square defined by coordinates of it's corners (sw, nw, ne, se).
grid_size = 10000 # meters
grid_unit = 1000 # meters 


def generate_grid(start_lat, start_lon, grid_size, grid_unit):
    # Get the grid's SW corner starting from the lj center coordinates
    grid_sw_corner = calculate_geo_point(start_lat, start_lon, 225, grid_size*math.sqrt(2)/2)

    # Calculate the rest of the grid's corners
    grid_corners = calculate_square_corners(grid_sw_corner[0], grid_sw_corner[1], grid_size)

    # generate the grid
    # start at the SW corner of the grid and make your way up 
    grid=[]

    row_origin = grid_sw_corner 
    for i in range(0, math.floor(grid_size/grid_unit)):
        col_origin = row_origin # remember the origin
        for j in range(0, math.floor(grid_size/grid_unit)):
            square = calculate_square_corners(col_origin[0], col_origin[1], grid_unit)
            grid.append(square)
            col_origin = square[3] # new SW corner is the SE corner of the previous square
    
        row_origin = calculate_geo_point(row_origin[0], row_origin[1], 0, grid_unit)
    
    return grid

In [28]:
len(generate_grid(lj_center_lat, lj_center_lon, grid_size, grid_unit))

100